In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

import keras
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import preprocess_input

from resnet import get_resnet

Using TensorFlow backend.


In [3]:
data_dir = '/home/ubuntu/data/'

In [4]:
data_generator = ImageDataGenerator(
    data_format='channels_last',
    preprocessing_function=lambda x: preprocess_input(np.expand_dims(x, 0))
)

train_generator = data_generator.flow_from_directory(
    data_dir + 'train', 
    target_size=(224, 224),
    batch_size=64
)

val_generator = data_generator.flow_from_directory(
    data_dir + 'val', 
    target_size=(224, 224),
    batch_size=64
)

Found 25600 images belonging to 256 classes.
Found 5120 images belonging to 256 classes.


In [5]:
model = get_resnet()

for layer in model.layers[:-14]:
    layer.trainable = False
    
model.layers[-2].kernel_regularizer = keras.regularizers.l2(1e-3)
model.layers[-8].kernel_regularizer = keras.regularizers.l2(1e-3)
model.layers[-11].kernel_regularizer = keras.regularizers.l2(1e-3)
model.layers[-14].kernel_regularizer = keras.regularizers.l2(1e-3)

In [6]:
model.compile(
    optimizer=optimizers.Adam(lr=1e-4), 
    loss='categorical_crossentropy', metrics=['accuracy', 'top_k_categorical_accuracy']
)

In [7]:
model.fit_generator(
    train_generator, 
    steps_per_epoch=150, epochs=7, verbose=1,
    validation_data=val_generator, validation_steps=24, 
    max_queue_size=10, workers=1, use_multiprocessing=False
)

Epoch 1/7
150/150 [==============================] - 121s - loss: 4.0736 - acc: 0.2687 - top_k_categorical_accuracy: 0.4311 - val_loss: 1.7483 - val_acc: 0.6113 - val_top_k_categorical_accuracy: 0.8164
Epoch 2/7
150/150 [==============================] - 119s - loss: 1.7099 - acc: 0.6953 - top_k_categorical_accuracy: 0.8689 - val_loss: 1.1489 - val_acc: 0.7357 - val_top_k_categorical_accuracy: 0.8978
Epoch 3/7
150/150 [==============================] - 119s - loss: 1.0084 - acc: 0.8169 - top_k_categorical_accuracy: 0.9393 - val_loss: 1.0904 - val_acc: 0.7604 - val_top_k_categorical_accuracy: 0.9108
Epoch 4/7
150/150 [==============================] - 119s - loss: 0.6345 - acc: 0.8904 - top_k_categorical_accuracy: 0.9709 - val_loss: 0.9540 - val_acc: 0.7760 - val_top_k_categorical_accuracy: 0.9316
Epoch 5/7
150/150 [==============================] - 119s - loss: 0.5164 - acc: 0.9061 - top_k_categorical_accuracy: 0.9793 - val_loss: 0.8660 - val_acc: 0.8047 - val_top_k_categorical_accurac

In [8]:
val_generator_no_shuffle = data_generator.flow_from_directory(
    data_dir + 'val', 
    target_size=(224, 224),
    batch_size=64, shuffle=False
)

Found 5120 images belonging to 256 classes.


In [9]:
model.evaluate_generator(val_generator_no_shuffle, 80)

[0.84789355546236034, 0.80488281250000004, 0.93398437499999998]

In [10]:
model.save_weights('resnet_weights.hdf5')